 
# This notebook comes as a helping hand to anyone interested in building or learning how to build a **movie recommender system using AutoEncoders**! Enjoy!

We are going to use 
* NumPy
* Pandas

and **PyTorch** for building the SAE (Sparse AutoEncoder)
    More information about Sparse Autoencoders can be found [here](https://web.stanford.edu/class/cs294a/sparseAutoencoder.pdf).

# Importing libraries

In [1]:
# importing libraries
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing dataset

In [2]:
# importing dataset
movies = pd.read_csv('/kaggle/input/movie-recommender-dataset/ml-1m/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/kaggle/input/movie-recommender-dataset/ml-1m/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/kaggle/input/movie-recommender-dataset/ml-1m/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Creating train and test sets

In [3]:
# creating train and test sets
training_set = pd.read_csv('/kaggle/input/movie-recommender-dataset/ml-100k/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('/kaggle/input/movie-recommender-dataset/ml-100k/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

# Getting number of users and movies

In [4]:
# getting the number of users and movies
nb_users = int(max(max(training_set[:, 0]), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))
print('no of users = ' + str(nb_users))
print('no of movies = ' + str(nb_movies))

no of users = 943
no of movies = 1682


# Converting the data into an array with users as lines and movies as columns

In [5]:
# converting the data into an array with users as lines and movies as columns
def conv(data):
    new_data = []
    for id_user in range(1, nb_users + 1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

# converting test and train sets
training_set = conv(training_set)
test_set = conv(test_set)

More information about Torch tensors and how they work can be found [here](https://pytorch.org/docs/stable/tensors.html).

# Converting the data into Torch tensors

In [6]:
# converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

More information about the **torch.nn** module can be found [here](https://pytorch.org/docs/stable/nn.html).

# Creating the arhitecture of the SAE

In [7]:
# creating the architecture of the Neural Network (SAE in our case is considered a directed neural network)
class SAE(nn.Module):
    def __init__(self):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

# Training the SAE

In [8]:
# training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data * mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: ' + str(epoch) + ' train_loss: ' + str(train_loss / s))

epoch: 1 train_loss: tensor(1.7712)
epoch: 2 train_loss: tensor(1.0968)
epoch: 3 train_loss: tensor(1.0533)
epoch: 4 train_loss: tensor(1.0381)
epoch: 5 train_loss: tensor(1.0310)
epoch: 6 train_loss: tensor(1.0265)
epoch: 7 train_loss: tensor(1.0238)
epoch: 8 train_loss: tensor(1.0220)
epoch: 9 train_loss: tensor(1.0210)
epoch: 10 train_loss: tensor(1.0194)
epoch: 11 train_loss: tensor(1.0190)
epoch: 12 train_loss: tensor(1.0183)
epoch: 13 train_loss: tensor(1.0180)
epoch: 14 train_loss: tensor(1.0174)
epoch: 15 train_loss: tensor(1.0174)
epoch: 16 train_loss: tensor(1.0167)
epoch: 17 train_loss: tensor(1.0169)
epoch: 18 train_loss: tensor(1.0164)
epoch: 19 train_loss: tensor(1.0163)
epoch: 20 train_loss: tensor(1.0161)
epoch: 21 train_loss: tensor(1.0161)
epoch: 22 train_loss: tensor(1.0160)
epoch: 23 train_loss: tensor(1.0159)
epoch: 24 train_loss: tensor(1.0159)
epoch: 25 train_loss: tensor(1.0158)
epoch: 26 train_loss: tensor(1.0155)
epoch: 27 train_loss: tensor(1.0153)
epoch: 28 

# Testing the SAE

In [9]:
# testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[(target == 0).unsqueeze(0)] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies / float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data * mean_corrector)
        s += 1.
        print('test loss: '+ str(test_loss / s))

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:445: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


test loss: tensor(1.0053)
test loss: tensor(0.9441)
test loss: tensor(1.0152)
test loss: tensor(0.8887)
test loss: tensor(0.9338)
test loss: tensor(0.9305)
test loss: tensor(0.9408)
test loss: tensor(0.9373)
test loss: tensor(0.9199)
test loss: tensor(0.8978)
test loss: tensor(0.8953)
test loss: tensor(0.8867)
test loss: tensor(0.9162)
test loss: tensor(0.9302)
test loss: tensor(0.9578)
test loss: tensor(0.9586)
test loss: tensor(0.9659)
test loss: tensor(0.9548)
test loss: tensor(0.9536)
test loss: tensor(0.9708)
test loss: tensor(0.9752)
test loss: tensor(0.9841)
test loss: tensor(0.9801)
test loss: tensor(0.9732)
test loss: tensor(0.9639)
test loss: tensor(0.9451)
test loss: tensor(0.9412)
test loss: tensor(0.9358)
test loss: tensor(0.9251)
test loss: tensor(0.9182)
test loss: tensor(0.9207)
test loss: tensor(0.9170)
test loss: tensor(0.9070)
test loss: tensor(0.9025)
test loss: tensor(0.9073)
test loss: tensor(0.9224)
test loss: tensor(0.9216)
test loss: tensor(0.9447)
test loss: t

The final loss on the test set is ~0.95 which proves to be an incredible result for our system!

*Thank you for your attention! :)*